In [1]:
import joblib
from joblib import Parallel, delayed
from multiprocessing import Queue
from itertools import product

from time import sleep

import subprocess

import os

import sys
sys.executable

'/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/bin/python3.8'

In [2]:
# Define number of GPUs available
NAME = 'LAMA_PRESET_PF_SMALL_TO_OLD'
PREFIX = 'runs'
#GPU_PARTS = [[0, 1], [2, 3]]
gpu = [0,1]
RUNNER_PATH = 'runners/lama_pf.py'
CONFIG_PATH = 'runners/lama_pf.yml'
DEBUG = True

TIMEOUT = 720
HARD_TIMEOUT = 720
NTHREADS = 4
SEED = 42

# benchmark_params
data_path = '../../data/old_presets/data'
benchmark_path = '../../data/old_presets'

In [3]:
# Put indices in queue
#q = Queue(maxsize=len(GPU_PARTS))
#for i in GPU_PARTS:
#    q.put(i)

data_info = joblib.load(os.path.join(benchmark_path, 'data_info.pkl'))

In [4]:
data_info

{'covertype': {'path': 'openml/covertype.csv',
  'target': 'class',
  'task_type': 'multiclass',
  'class_map': {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6},
  'read_csv_params': {'na_values': '?'}},
 'albert': {'path': 'openml/albert.csv',
  'target': 'class',
  'task_type': 'binary',
  'read_csv_params': {'na_values': '?'}},
 'higgs': {'path': 'openml/higgs.csv',
  'target': 'class',
  'task_type': 'binary',
  'read_csv_params': {'na_values': '?'}},
 'guillermo': {'path': 'openml/guillermo.csv',
  'target': 'class',
  'task_type': 'binary',
  'read_csv_params': {'na_values': '?'}},
 'bank-marketing': {'path': 'openml/bank-marketing.csv',
  'target': 'Class',
  'task_type': 'binary',
  'class_map': {1: 0, 2: 1},
  'read_csv_params': {'na_values': '?'}},
 'numerai28.6': {'path': 'openml/numerai28.6.csv',
  'target': 'attribute_21',
  'task_type': 'binary',
  'read_csv_params': {'na_values': '?'}},
 'volkert': {'path': 'openml/volkert.csv',
  'target': 'class',
  'task_type': 'multiclass'

In [5]:
def run_task(benchmark_path, data_path, dataset, fold, rewrite=False):
    #gpu = q.get()
    
    run_name = dataset + '_' + str(fold)
    
    print('Processing', run_name)
    
    benchmark_path = os.path.abspath(benchmark_path)
    data_path = os.path.abspath(data_path)
    output = os.path.join(benchmark_path, PREFIX, NAME, dataset, 'fold_{0}'.format(fold))
    
#     sleep(5)
    
    os.makedirs(output, exist_ok=True)
    
    success_flg = os.path.join(output, 'SUCCESS')
    
    #if os.path.exists(success_flg) and not rewrite:
    #    q.put(gpu)
    #    return 
    
    # clean folder
    for f in (x for x in os.listdir(output) if not x.startswith('.')):
        os.remove(os.path.join(output, f))
    
    # TRAIN
    try:
        
        script = "source /trinity/home/r.zagidullin/miniconda3/bin/activate; conda activate new_lama_venv; which python; "
        
        log = subprocess.check_output(script + ' '.join([
            'python', 
            RUNNER_PATH,
            '-b', benchmark_path,
            '-p', data_path,
            '-k', dataset,
            '-f', str(fold), 
            '-n', str(NTHREADS),
            '-s', str(SEED),
            '-d', ','.join(map(str, gpu)),
            '-c', os.path.abspath(CONFIG_PATH),
            '-t', str(TIMEOUT),
            '-o', output

        ]), timeout=HARD_TIMEOUT, shell=True, stderr=subprocess.STDOUT, executable='/bin/bash').decode()
        
        if DEBUG:
            print(log)
        
        with open(success_flg, 'w') as f:
            pass
        
        with open(os.path.join(output, 'train_log.txt'), 'w') as f:
            f.write(log)
    
    except subprocess.CalledProcessError as e:

        print(e.output.decode())
        
        with open(os.path.join(output, 'ERROR'), 'w') as f:
            pass
        
        with open(os.path.join(output, 'train_log.txt'), 'w') as f:
            f.write(e.output.decode())
                  
    except subprocess.TimeoutExpired:
        
        
        with open(os.path.join(output, 'TIMEOUT'), 'w') as f:
            pass
        
        print('HARD TIMEOUT!')
        
    #q.put(gpu)

    
    
    
# def run_task(benchmark_path, data_path, dataset, fold, rewrite=False):
    
#     gpu = q.get()
#     _run_task(benchmark_path, data_path, dataset, fold, gpu, rewrite)
#     q.put(gpu)
    
#     return 

In [11]:
run_task(benchmark_path, data_path, 'airlines', 4, True)

Processing airlines_4
/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/bin/python
Train dataset airlines, fold 4
  Airline  Flight AirportFrom AirportTo  DayOfWeek  Time  Length  Delay
0      CO     269         SFO       IAH          3    15     205      1
1      US    1558         PHX       CLT          3    15     222      1
2      AA    2400         LAX       DFW          3    20     165      1
3      AA    2466         SFO       DFW          3    20     195      1
4      AS     108         ANC       SEA          3    30     202      0
[13:35:41] Stdout logging level is DEBUG.
[13:35:41] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[13:35:41] Task: binary

[13:35:41] Start automl preset with listed constraints:
[13:35:41] - time: 720.00 seconds
[13:35:41] - CPU: 4 cores
[13:35:41] - memory: 16 GB

[13:35:41] Train data shape: (431507, 8)
Feats was rejected during automatic roles guess: []
[13:35:42] Layer 1 train process 

In [12]:
run_task(benchmark_path, data_path, 'ieee-fraud-detection', 4, True)

Processing ieee-fraud-detection_4
/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/bin/python
/bin/bash: line 1: 2277179 Killed                  python runners/lama_pf.py -b /beegfs/home/r.zagidullin/LAMA/LightAutoML/data/old_presets -p /beegfs/home/r.zagidullin/LAMA/LightAutoML/data/old_presets/data -k ieee-fraud-detection -f 4 -n 4 -s 42 -d 0,1 -c /beegfs/home/r.zagidullin/LAMA/LightAutoML/examples/old/runners/lama_pf.yml -t 720 -o /beegfs/home/r.zagidullin/LAMA/LightAutoML/data/old_presets/runs/LAMA_PRESET_PF_SMALL_TO_OLD/ieee-fraud-detection/fold_4



In [ ]:
# Parallel(n_jobs=len(GPU_PARTS), backend="threading")(
#     delayed(run_task)(benchmark_path, d, f, False) for (d, s) in product(data_info, range(10)))

In [ ]:
Parallel(n_jobs=len(GPU_PARTS), backend="threading")(
    delayed(run_task)(benchmark_path, data_path, d, f, False) for (d, f) in product(data_info, range(1)))

Processing covertype_0
Processing albert_0
/home/user/conda/envs/rapids-21.08/bin/python
Train dataset albert, fold 0
   class    V1    V2    V3    V4       V5  ...  V73  V74   V75  V76   V77   V78
0      1   2.0   0.0   9.0   5.0      0.0  ...   43   43   1.0  103   870   662
1      1   NaN  -1.0   NaN   NaN   6736.0  ...   43   43   6.0  188  8016  1887
2      0   1.0   0.0  11.0   4.0     20.0  ...   43   43   2.0   57  4740  1035
3      0  10.0   0.0  33.0  37.0      1.0  ...  103   43  19.0  103  4741  1512
4      0   NaN  24.0   5.0   9.0  13632.0  ...   43   43   1.0   43  5454  1138

[5 rows x 79 columns]
setting gpuids with all
[14:12:23] Stdout logging level is DEBUG.
[14:12:23] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[14:12:23] Task: binary

[14:12:23] Start automl preset with listed constraints:
[14:12:23] - time: 720.00 seconds
[14:12:23] - CPU: 4 cores
[14:12:23] - memory: 16 GB

READER:
  output is: mgpu
  num_gp

HARD TIMEOUT!
Processing bank-marketing_0
/home/user/conda/envs/rapids-21.08/bin/python
Train dataset bank-marketing, fold 0
   V1            V2       V3         V4  V5  ...  V13 V14 V15      V16  Class
0  58    management  married   tertiary  no  ...    1  -1   0  unknown      0
1  44    technician   single  secondary  no  ...    1  -1   0  unknown      0
2  33  entrepreneur  married  secondary  no  ...    1  -1   0  unknown      0
3  47   blue-collar  married    unknown  no  ...    1  -1   0  unknown      0
4  33       unknown   single    unknown  no  ...    1  -1   0  unknown      0

[5 rows x 17 columns]
setting gpuids with all
[14:24:20] Stdout logging level is DEBUG.
[14:24:20] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[14:24:20] Task: binary

[14:24:20] Start automl preset with listed constraints:
[14:24:20] - time: 720.00 seconds
[14:24:20] - CPU: 4 cores
[14:24:20] - memory: 16 GB

READER:
  output is: mgpu
  num_gpu_rea

/home/user/conda/envs/rapids-21.08/bin/python
Train dataset numerai28.6, fold 0
   attribute_0  attribute_1  ...  attribute_20  attribute_21
0     0.497449     0.722213  ...      0.639446             0
1     0.314430     0.442740  ...      0.216417             1
2     0.542753     0.749818  ...      0.985990             1
3     0.789343     0.624176  ...      0.664026             0
4     0.046066     0.815840  ...      0.217831             1

[5 rows x 22 columns]
setting gpuids with all
[14:27:38] Stdout logging level is DEBUG.
[14:27:38] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[14:27:38] Task: binary

[14:27:38] Start automl preset with listed constraints:
[14:27:38] - time: 720.00 seconds
[14:27:38] - CPU: 4 cores
[14:27:38] - memory: 16 GB

READER:
  output is: mgpu
  num_gpu_readers: 2
  num_cpu_readers: 4
[14:27:47] Train data shape: (86688, 22)
daskcudf reader: 0.38743586651980877
hybrid reader: 9.28884949348867
[14:27:4

/home/user/conda/envs/rapids-21.08/bin/python
Train dataset adult, fold 0
   age   workclass  fnlwgt  ... hours-per-week  native-country class
0   25     Private  226802  ...             40   United-States     0
1   38     Private   89814  ...             50   United-States     0
2   28   Local-gov  336951  ...             40   United-States     1
3   44     Private  160323  ...             40   United-States     1
4   18           ?  103497  ...             30   United-States     0

[5 rows x 15 columns]
setting gpuids with all
[14:29:25] Stdout logging level is DEBUG.
[14:29:25] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[14:29:25] Task: binary

[14:29:25] Start automl preset with listed constraints:
[14:29:25] - time: 720.00 seconds
[14:29:25] - CPU: 4 cores
[14:29:25] - memory: 16 GB

READER:
  output is: mgpu
  num_gpu_readers: 2
  num_cpu_readers: 4
[14:29:33] Train data shape: (43957, 15)
daskcudf reader: 0.4879192840307951

/home/user/conda/envs/rapids-21.08/bin/python
Train dataset MiniBooNE, fold 0
   signal  ParticleID_0  ...  ParticleID_48  ParticleID_49
0       1       2.59413  ...       0.071769       0.245996
1       1       3.86388  ...       0.333613       0.230621
2       1       3.38584  ...       0.255512       0.180901
3       1       4.28524  ...       0.473081       0.258990
4       1       5.93662  ...       1.924990       0.253893

[5 rows x 51 columns]
setting gpuids with all
[14:32:27] Stdout logging level is DEBUG.
[14:32:27] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[14:32:27] Task: binary

[14:32:27] Start automl preset with listed constraints:
[14:32:27] - time: 720.00 seconds
[14:32:27] - CPU: 4 cores
[14:32:27] - memory: 16 GB

READER:
  output is: mgpu
  num_gpu_readers: 2
  num_cpu_readers: 4
[14:32:37] Train data shape: (117057, 51)
daskcudf reader: 1.1639997903257608
hybrid reader: 10.791778171434999
[14:32:38] Layer 1 t

/home/user/conda/envs/rapids-21.08/bin/python
Train dataset shuttle, fold 0
   A1  A2  A3  A4  A5  A6  A7  A8  A9  class
0  50  21  77   0  28   0  27  48  22      1
1  55   0  92   0   0  26  36  92  56      3
2  53   0  82   0  52  -5  29  30   2      0
3  37   0  76   0  28  18  40  48   8      0
4  37   0  79   0  34 -26  43  46   2      0
setting gpuids with all
[14:45:34] Stdout logging level is DEBUG.
[14:45:34] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[14:45:34] Task: multiclass

[14:45:34] Start automl preset with listed constraints:
[14:45:34] - time: 720.00 seconds
[14:45:34] - CPU: 4 cores
[14:45:34] - memory: 16 GB

READER:
  output is: mgpu
  num_gpu_readers: 2
  num_cpu_readers: 4
[14:45:48] Train data shape: (52200, 10)
daskcudf reader: 0.35292075388133526
hybrid reader: 15.016378089785576
[14:45:49] Layer 1 train process start. Time left 704.97 secs
[14:45:59] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[14:46

HARD TIMEOUT!
Processing jannis_0
Processing nomao_0
/home/user/conda/envs/rapids-21.08/bin/python
Train dataset nomao, fold 0
    V1    V2        V3        V4  ...  V116  V117      V118  Class
0  1.0  1.00  1.000000  1.000000  ...     2   1.0  1.000000      1
1  1.0  0.75  0.857143  0.857143  ...     2   1.0  1.000000      1
2  1.0  1.00  1.000000  1.000000  ...     2   1.0  1.000000      1
3  1.0  0.75  0.857143  0.857143  ...     2   1.0  0.999994      1
4  0.0  0.00  0.250000  0.000000  ...     1   1.0  0.979322      1

[5 rows x 119 columns]
setting gpuids with all
[15:00:27] Stdout logging level is DEBUG.
[15:00:27] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[15:00:27] Task: binary

[15:00:27] Start automl preset with listed constraints:
[15:00:27] - time: 720.00 seconds
[15:00:27] - CPU: 4 cores
[15:00:27] - memory: 16 GB

READER:
  output is: mgpu
  num_gpu_readers: 2
  num_cpu_readers: 4
Traceback (most recent call last):